In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import regionmask
import pandas as pd
from geopy.geocoders import Nominatim
from matplotlib.colors import SymLogNorm
from matplotlib.pyplot import cm

import xesmf as xe
#from pykrige.ok import OrdinaryKriging
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import dask
import utils


import cartopy.feature
from cartopy.mpl.patch import geos_to_path
import cartopy.crs as ccrs

import geopandas

In [2]:
## look at https://wedocs.unep.org/bitstream/handle/20.500.11822/11406/Pog_&amp;_iPog_-_Energy_and_Fuels.pdf?sequence=1&amp%3BisAllowed=

## for emissions factors scaling

## fix tott

## Set constants

In [3]:
coal_year_range = np.arange(0,41)
percents = [0, 10, 20,30,40,50,60, 70,80,90, 99]
years = [0,4,10,16,20,24,30,36]
coal_year_range = np.arange(0,41)

In [4]:
## Add time dimension
length_simulation = 50*365

time_array = np.arange(0, length_simulation)

## Data read in

### Emissions dataframe

In [5]:
CGP_df = pd.read_csv('mod_coal_inputs/BC_limited_country_SEA_GAINS_Springer.csv')

In [6]:
CGP_df.columns = CGP_df.columns.str.replace(' ', '_')

In [7]:
CGP_df = CGP_df.rename(columns = {'YEAR':'Year_of_Commission', 'EMISFACTOR.PLATTS':'CO2_weighted_capacity_1000tonsperMW'})

### Convolution

In [8]:
da = {}
for yr in years:
    print(yr)
    da[yr] = {}
    for pc in percents:
        print(yr, pc)
        da[yr][pc] = xr.open_mfdataset(f'Outputs/C_out_{pc}_{yr}.nc', chunks = 'auto')
        
da2 = {}
for pc in percents:
    da2[pc] = xr.concat([da[yr][pc] for yr in years], pd.Index([yr for yr in years], name = 'year_shutdown'))
    
ds = xr.concat([da2[pc] for pc in percents], pd.Index([pc for pc in percents], name = 'percent'))
ds = ds.rename({'__xarray_dataarray_variable__':'BC_conc'})
ds['BC_conc']*=1e9
ds['BC_conc'].attrs = {'units':'ppb'}

0
0 0
0 10
0 20
0 30
0 40
0 50
0 60
0 70
0 80
0 90
0 99
4
4 0
4 10
4 20
4 30
4 40
4 50
4 60
4 70
4 80
4 90
4 99
10
10 0
10 10
10 20
10 30
10 40
10 50
10 60
10 70
10 80
10 90
10 99
16
16 0
16 10
16 20
16 30
16 40
16 50
16 60
16 70
16 80
16 90
16 99
20
20 0
20 10
20 20
20 30
20 40
20 50
20 60
20 70
20 80
20 90
20 99
24
24 0
24 10
24 20
24 30
24 40
24 50
24 60
24 70
24 80
24 90
24 99
30
30 0
30 10
30 20
30 30
30 40
30 50
30 60
30 70
30 80
30 90
30 99
36
36 0
36 10
36 20
36 30
36 40
36 50
36 60
36 70
36 80
36 90
36 99


In [9]:
def ppb_to_ug(ds, species_to_convert, mw_species_list, stp_p = 101325, stp_t = 298.):
    '''Convert species to ug/m3 from ppb'''
    R = 8.314 #J/K/mol
    ppb_ugm3 = (stp_p / stp_t / R) #Pa/K/(J/K/mol) = g/m^3

    for spec in species_to_convert:
        attrs = ds[spec].attrs
        ds[spec] = ds[spec]*mw_species_list[spec]*ppb_ugm3 #ppb*g/mol*g/m^3
        ds[spec].attrs['units'] = 'μg m-3'
    #return(ds)

mw_BC = {'BC_conc':12.011}
ppb_to_ug(ds, ['BC_conc'], mw_BC)

In [10]:
ds['percent'] = 100- ds['percent']

### Area for weighting

In [11]:
ds_area = xr.open_dataset('/net/fs11/d0/emfreese/GCrundirs/IRF_runs/stretch_2x_pulse/SEA/Jan/mod_output/GEOSChem.SpeciesConc.20160101_0000z.nc4', engine = 'netcdf4')

In [12]:
ds_area = utils.fix_area_ij_latlon(ds_area)

## Country Mask

In [15]:
country_mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110
countries = ['China','Indonesia','Malaysia','Vietnam','Australia', 'Cambodia','Myanmar', 'Laos','Philippines','Nepal','Bangladesh','Thailand','Bhutan']
print('countries uploaded')

countries uploaded


## Dataframes by country exposure (sum and mean)

In [16]:
percents = [100-p for p in percents]

In [17]:
percents

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [31]:
mask = {}
contiguous_mask = {}
mask_ds = {}
for idx, c in enumerate(countries):
    mask[c] = country_mask.mask(ds, lon_name = 'lon', lat_name = 'lat')
    contiguous_mask[c] = ~np.isnan(mask[c] )& (mask[c]  == country_mask.map_keys(countries[idx] ))
    mask_ds[c] = ds.where(contiguous_mask[c]).weighted(ds_area['area']).mean(dim = ['lat','lon'])#.sum(dim = ['s'])


In [33]:
mask_full_ds = xr.concat([mask_ds[c] for c in countries], pd.Index([c for c in countries], name = 'country'))

In [47]:
years = pd.date_range(f"{int(CGP_df['Year_of_Commission'].min())}-01-01", freq="D", periods=len(mask_full_ds['s']))

In [49]:
mask_full_ds['s'] = years

In [52]:
mask_annual_mean_ds = mask_full_ds.groupby('s.year').mean()

In [54]:
mask_annual_mean_ds.to_netcdf(f'Outputs/multi_country_early_shutdown_annual_means.nc4')

In [55]:
xr.Dataset.to_zarr(mask_annual_mean_ds, f'Outputs/multi_country_early_shutdown_annual_means.zarr', mode = 'w') #save the dataset 


NameError: name 'zarr' is not defined